In [1]:
#

In [2]:
import json
import time
import numpy
import pandas
import datetime

In [3]:
from sklearn.svm import NuSVR as NVR

In [4]:
from thats_the_stuff import golags

In [5]:
# models
# all of them are stored in .json file with possible parameters

import json

with open('./models_params_mini.json') as f:
    models_params = json.load(f)

In [6]:
def multiply_params(params):
    keys = numpy.array(list(params.keys()))
    dims = numpy.array([len(params[keys[j]]) for j in numpy.arange(keys.shape[0])])
    result = []
    for j in numpy.arange(dims.prod()):
        curr = j
        res = {}
        for k in numpy.arange(keys.shape[0]):
            ix = curr % dims[k]
            res[keys[k]] = params[keys[k]][ix]
            curr = curr // dims[k]
        result.append(res)
    return result


In [7]:
dim_models = 0
multiple_model_args = multiply_params(models_params['NuSVR'])
multiple_model_args

[{'nu': 0.1,
  'C': 0.1,
  'kernel': 'poly',
  'gamma': 'auto',
  'shrinking': True,
  'max_iter': 10000},
 {'nu': 0.5,
  'C': 0.1,
  'kernel': 'poly',
  'gamma': 'auto',
  'shrinking': True,
  'max_iter': 10000},
 {'nu': 1.0,
  'C': 0.1,
  'kernel': 'poly',
  'gamma': 'auto',
  'shrinking': True,
  'max_iter': 10000},
 {'nu': 0.1,
  'C': 1.0,
  'kernel': 'poly',
  'gamma': 'auto',
  'shrinking': True,
  'max_iter': 10000},
 {'nu': 0.5,
  'C': 1.0,
  'kernel': 'poly',
  'gamma': 'auto',
  'shrinking': True,
  'max_iter': 10000},
 {'nu': 1.0,
  'C': 1.0,
  'kernel': 'poly',
  'gamma': 'auto',
  'shrinking': True,
  'max_iter': 10000},
 {'nu': 0.1,
  'C': 10.0,
  'kernel': 'poly',
  'gamma': 'auto',
  'shrinking': True,
  'max_iter': 10000},
 {'nu': 0.5,
  'C': 10.0,
  'kernel': 'poly',
  'gamma': 'auto',
  'shrinking': True,
  'max_iter': 10000},
 {'nu': 1.0,
  'C': 10.0,
  'kernel': 'poly',
  'gamma': 'auto',
  'shrinking': True,
  'max_iter': 10000},
 {'nu': 0.1,
  'C': 0.1,
  'kernel

In [8]:
d = './dataset.csv'
data = pandas.read_csv(d)
data = data.rename(columns={'lag': 'news_horizon'})
data = data.set_index(['ticker', 'time', 'news_horizon'], drop=False)
data = data.sort_index()
data

ticker  \
ticker time                      news_horizon          
MSFT   2019-08-30 15:08:00+00:00 3              MSFT   
                                 3              MSFT   
                                 6              MSFT   
                                 6              MSFT   
                                 9              MSFT   
...                                              ...   
       2019-09-26 20:00:00+00:00 94             MSFT   
                                 94             MSFT   
                                 94             MSFT   
                                 94             MSFT   
                                 97             MSFT   

                                                                    time  \
ticker time                      news_horizon                              
MSFT   2019-08-30 15:08:00+00:00 3             2019-08-30 15:08:00+00:00   
                                 3             2019-08-30 15:08:00+00:00   
                                 6             2019-08-30 15:08:00+00:00   
                                 6             2019-08-30 15:08:00+00:00   
                                 9             2019-08-30 15:08:00+00:00   
...                                                                  ...   
       2019-09-26 20:00:00+00:00 94            2019-09-26 20:00:00+00:00   
                                 94            2019-09-26 20:00:00+00:00   
                                 94            2019-09-26 20:00:00+00:00   
                                 94            2019-09-26 20:00:00+00:00   
                                 97            2019-09-26 20:00:00+00:00   

                                               open_LAG0  close_LAG0  \
ticker time                      news_horizon                          
MSFT   2019-08-30 15:08:00+00:00 3             -0.000292   -0.000438   
                                 3             -0.000292   -0.000438   
                                 6             -0.000292   -0.000438   
                                 6             -0.000292   -0.000438   
                                 9             -0.000292   -0.000438   
...                                                  ...         ...   
       2019-09-26 20:00:00+00:00 94            -0.000072    0.000716   
                                 94            -0.000072    0.000716   
                                 94            -0.000072    0.000716   
                                 94            -0.000072    0.000716   
                                 97            -0.000072    0.000716   

                                               high_LAG0  low_LAG0  \
ticker time                      news_horizon                        
MSFT   2019-08-30 15:08:00+00:00 3             -0.000365 -0.000511   
                                 3             -0.000365 -0.000511   
                                 6             -0.000365 -0.000511   
                                 6             -0.000365 -0.000511   
                                 9             -0.000365 -0.000511   
...                                                  ...       ...   
       2019-09-26 20:00:00+00:00 94            -0.000215  0.000933   
                                 94            -0.000215  0.000933   
                                 94            -0.000215  0.000933   
                                 94            -0.000215  0.000933   
                                 97            -0.000215  0.000933   

                                               volume_LAG0  open_LAG1  \
ticker time                      news_horizon                           
MSFT   2019-08-30 15:08:00+00:00 3                0.539497  -0.000146   
                                 3                0.539497  -0.000146   
                                 6                0.539497  -0.000146   
                                 6                0.539497  -0.000146   
                                 9        

In [ ]:
tsi_names = ['news_time']
y_names = ['open_LAG0']
removes = ['ticker', 'time', 'id', 'title', 'news_time']

report = golags(NVR, data, multiple_model_args, tsi_names, y_names, removes, n_folds=5)
report

Fold shape: (21483, 501)
Test shape: (36978, 501)
N of expected iters = 54
Started search: 2020-06-15T23:16:42.265896
0 / 54
